# Model Deployment

To run this notebook set the GOOGLE CREDENTIAL PATH

In [0]:
%pip install google-auth google-cloud-storage google-cloud-mlflow databricks-registry-webhooks -q 

Python interpreter will be restarted.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-cpu 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.
Python interpreter will be restarted.


## Import libraries

In [0]:
# General
import random
import string
import os
import urllib
import time

# Deployment
import mlflow
from  mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import google.oauth2.id_token
import google.auth.transport.requests
from databricks_registry_webhooks import RegistryWebhooksClient, HttpUrlSpec

## Define constants

In [0]:
# General
UUID = "2obn" #Indicate the UUID created 
PROJECT_ID = "leedeb-experimentation"
REGION = "us-central1"
BUCKET_NAME = "vertex-ai-databricks-retail-demo"
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [0]:
# Model Deployment 
CLOUD_FUNCTION_NAME = "deployv2"
CLOUD_FUNCTION_ENTRY_POINT = "deploy"
# CLOUD_FUNCTION_URL = f"https://{REGION}-{PROJECT_ID}.cloudfunctions.net/{CLOUD_FUNCTION_NAME}"
CLOUD_FUNCTION_URL = f"https://{CLOUD_FUNCTION_NAME}-witrkzd5va-uc.a.run.app/{CLOUD_FUNCTION_ENTRY_POINT}"

EXPERIMENT_NAME = "training"
MODEL_NAME = "Weekly_Sales_GBTR_model_" + UUID
REGISTERED_MODEL = "databricks_model2"
MODEL_STAGE="production"

## Helpers

In [0]:
# auth function
def make_authorized_get_request(endpoint, audience):
    """
    Make an authorized request to the given endpoint.
    Args:
        endpoint: The endpoint to send the request to.
        audience: The audience to use when validating the JWT.
    Returns:
        The JSON response from the request.
    """
    # Define the request.
    req = urllib.request.Request(endpoint)

    # Get the ID token from the environment.
    auth_req = google.auth.transport.requests.Request()
    id_token = google.oauth2.id_token.fetch_id_token(auth_req, audience)
    
    return id_token

## Register the model

In [0]:
client = MlflowClient()

In [0]:
last_experiment = client.search_experiments()[0]
last_run = client.search_runs(last_experiment.experiment_id)[0]
last_run_id = last_run.info.run_id

In [0]:
last_run_id

Out[41]: '4ec16154d809480ea7db87dacfbfee33'

In [0]:
model_uri = f"runs:/{last_run_id}/{MODEL_NAME}"
registered_model = mlflow.register_model(model_uri=model_uri, name=REGISTERED_MODEL)
time.sleep(20)

Successfully registered model 'databricks_model2'.
2023/02/02 07:28:33 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: databricks_model2, version 1
Created version '1' of model 'databricks_model2'.


In [0]:
# add detailed comments for this specific model version that being registered
client.update_model_version(
  name=registered_model.name,
  version=registered_model.version,
  description="GBTRegressor MLLib PySpark trained for streaming pipeline, fixed issues with staging to production."
)

Out[43]: <ModelVersion: creation_timestamp=1675322913329, current_stage='None', description=('GBTRegressor MLLib PySpark trained for streaming pipeline, fixed issues with '
 'staging to production.'), last_updated_timestamp=1675322946278, name='databricks_model2', run_id='4ec16154d809480ea7db87dacfbfee33', run_link='', source='dbfs:/databricks/mlflow-tracking/bcd044c1237440d7866d3363fd45fffb/4ec16154d809480ea7db87dacfbfee33/artifacts/Weekly_Sales_GBTR_model_2obn', status='READY', status_message='', tags={}, user_id='3114545449803070', version='1'>

## Create the HTTP registry webhook

Webhooks enable you to listen for Model Registry events so your integrations can automatically trigger actions. You can use webhooks to automate and integrate your machine learning pipeline with existing CI/CD tools and workflows. 

For example, in our case we are going to pass the event in a cloud function and we will trigger a CI/CD builds to deploy the model on Vertex AI.

### Get the auth to call the function from the webhook

In [0]:
token_id  = make_authorized_get_request(CLOUD_FUNCTION_URL, CLOUD_FUNCTION_URL)

In [0]:
http_url_spec = HttpUrlSpec(
  url=CLOUD_FUNCTION_URL,
  authorization=f"Bearer {token_id}"
)
http_webhook = RegistryWebhooksClient().create_webhook(
  model_name=registered_model.name, 
  events=["MODEL_VERSION_TRANSITIONED_STAGE"],
  http_url_spec=http_url_spec,
  description="Testing deploy model",
  status="TEST_MODE"
)


In [0]:
http_webhook

Out[48]: <RegistryWebhook: creation_timestamp=1675322948311, description='Testing deploy model', events=['MODEL_VERSION_TRANSITIONED_STAGE'], http_url_spec=<HttpUrlSpec: authorization=None, enable_ssl_verification=True, secret=None, url='https://deployv2-witrkzd5va-uc.a.run.app/deploy'>, id='1405f1b7a86940e0b185b270a288c4f2', job_spec=None, last_updated_timestamp=1675322948311, model_name='databricks_model2', status='TEST_MODE'>

### Update the webhook to active status
To enable the webhook for real events, set its status to ACTIVE through an update call, which can also be used to change any of its other properties.

In [0]:
http_webhook = RegistryWebhooksClient().update_webhook(
  id=http_webhook.id,
  status="ACTIVE"
)

In [0]:
http_webhook

Out[50]: <RegistryWebhook: creation_timestamp=1675322948311, description='Testing deploy model', events=['MODEL_VERSION_TRANSITIONED_STAGE'], http_url_spec=<HttpUrlSpec: authorization=None, enable_ssl_verification=True, secret=None, url='https://deployv2-witrkzd5va-uc.a.run.app/deploy'>, id='1405f1b7a86940e0b185b270a288c4f2', job_spec=None, last_updated_timestamp=1675322949097, model_name='databricks_model2', status='ACTIVE'>

## Transition model to production stage

In [0]:
client.transition_model_version_stage(
  name=registered_model.name,
  version=registered_model.version,
  stage=MODEL_STAGE,
)

Out[51]: <ModelVersion: creation_timestamp=1675322913329, current_stage='Production', description=('GBTRegressor MLLib PySpark trained for streaming pipeline, fixed issues with '
 'staging to production.'), last_updated_timestamp=1675322949408, name='databricks_model2', run_id='4ec16154d809480ea7db87dacfbfee33', run_link='', source='dbfs:/databricks/mlflow-tracking/bcd044c1237440d7866d3363fd45fffb/4ec16154d809480ea7db87dacfbfee33/artifacts/Weekly_Sales_GBTR_model_2obn', status='READY', status_message='', tags={}, user_id='3114545449803070', version='1'>

## Do not run

In [0]:
webhooks_list = RegistryWebhooksClient().list_webhooks(
  events=["MODEL_VERSION_TRANSITIONED_STAGE"]
)

In [0]:
for webhook in webhooks_list:
  RegistryWebhooksClient().delete_webhook(webhook.id)